# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import datetime
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vac_df = pd.read_csv("../../output_data/cityweather_data.csv")
vac_df.head()

,City,Lat,Lng,Max temp,Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,Kavieng,-2.5744,150.7967,80.69,80.69,82,100,7.56,PG,1639820794
1,Saint-Philippe,-21.3585,55.7679,86.11,83.97,72,16,2.01,RE,1639821072
2,Yulara,-25.2406,130.9889,109.24,109.24,8,15,10.36,AU,1639820871
3,Hermanus,-34.4187,19.2345,89.91,78.80,42,10,13.42,ZA,1639821073
4,Khatanga,71.9667,102.5000,-39.32,-39.32,100,65,3.13,RU,1639820909


In [3]:
vac_df.shape

(541, 10)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations
vlocations = vac_df[["Lat","Lng"]]

In [5]:
# Store humidity weight values and convert to float
hum = vac_df["Humidity"].astype(float)
hum

0       82.0
1       72.0
2        8.0
3       42.0
4      100.0
       ...  
536     33.0
537     99.0
538    100.0
539     75.0
540     62.0
Name: Humidity, Length: 541, dtype: float64

In [6]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(vlocations, weights=hum, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

* Narrow down the DataFrame to find your ideal weather condition. For example:
* A max temperature lower than 80 degrees but higher than 70.
* Wind speed less than 10 mph.
* Zero cloudiness.
* Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
* **Note:** Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [7]:
Tindex = vac_df[(vac_df['Max temp']>=70) & (vac_df['Max temp']<=80) & (vac_df['Windspeed']<=10) & (vac_df['Cloudiness']<=0)].index
Tindex

Int64Index([28, 82, 186, 296, 307, 328, 362, 491, 503], dtype='int64')

In [8]:
hotel_df = vac_df.drop(Tindex,inplace=False)
hotel_df

,City,Lat,Lng,Max temp,Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,Kavieng,-2.5744,150.7967,80.69,80.69,82,100,7.56,PG,1639820794
1,Saint-Philippe,-21.3585,55.7679,86.11,83.97,72,16,2.01,RE,1639821072
2,Yulara,-25.2406,130.9889,109.24,109.24,8,15,10.36,AU,1639820871
3,Hermanus,-34.4187,19.2345,89.91,78.80,42,10,13.42,ZA,1639821073
4,Khatanga,71.9667,102.5000,-39.32,-39.32,100,65,3.13,RU,1639820909
...,...,...,...,...,...,...,...,...,...,...
536,Saryozek,44.3583,77.9753,40.93,40.93,33,64,3.04,KZ,1639821777
537,Batagay-Alyta,67.8006,130.4114,-23.31,-23.31,99,100,1.39,RU,1639821777
538,Morehead,37.2711,-87.1764,58.98,52.92,100,90,0.00,US,1639821777
539,Belyy Yar,53.6039,91.3903,33.69,29.86,75,0,11.18,RU,1639821778


# Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Lat,Lng,Max temp,Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
0,Kavieng,-2.5744,150.7967,80.69,80.69,82,100,7.56,PG,1639820794,
1,Saint-Philippe,-21.3585,55.7679,86.11,83.97,72,16,2.01,RE,1639821072,
2,Yulara,-25.2406,130.9889,109.24,109.24,8,15,10.36,AU,1639820871,
3,Hermanus,-34.4187,19.2345,89.91,78.80,42,10,13.42,ZA,1639821073,
4,Khatanga,71.9667,102.5000,-39.32,-39.32,100,65,3.13,RU,1639820909,


In [11]:
# params dictionary to update each iteration

params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}
               
# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
        # get lat, lng from df
        lat = row["Lat"]
        lng = row["Lng"]
        city = row["City"]
        # change location each iteration while leaving original params in place
        params["location"] = f"{lat},{lng}"
        
        #print(f"Retrieving nearest Hotel to City {index}: {city}.")
        # Use the search term: "hotel" and our lat/lng
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        response = requests.get(base_url, params=params)
        name_hotel = response.json()
        # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
        try:
            print(f"{index} Closest {city} hotel is {name_hotel['results'][0]['name']}.")
            hotel_df.loc[index, "Hotel Name"] = name_hotel['results'][0]["name"]
        except (KeyError, IndexError):
            print(f"Missing field...skipping {index} {city}")
            pass           

0 Closest Kavieng hotel is Kavieng.
1 Closest Saint-Philippe hotel is Saint-Philippe.
2 Closest Yulara hotel is Petermann.
3 Closest Hermanus hotel is Hermanus.
4 Closest Khatanga hotel is Chatanga.
5 Closest Trstená hotel is Trstená.
6 Closest Takaungu hotel is Kilifi.
7 Closest Mataura hotel is Gore.
8 Closest Saint Paul Harbor hotel is Kodiak.
9 Closest Newport hotel is Newport.
10 Closest Melilla hotel is Melilla.
11 Closest Punta Arenas hotel is Punta Arenas.
12 Closest Aklavik hotel is Aklavik.
13 Closest Carnot hotel is Carnot.
14 Closest Chui hotel is Chuy.
15 Closest Hithadhoo hotel is Addu City.
16 Closest Guane hotel is Guane.
17 Closest Salinópolis hotel is Salinópolis.
18 Closest Vaini hotel is Nukuhetulu.
19 Closest Bethel hotel is Danbury.
20 Closest Rikitea hotel is Rikitea.
21 Closest Tirukkoyilur hotel is Thirukoilure.
22 Closest Geraldton hotel is Geraldton.
23 Closest Avarua hotel is Avarua District.
24 Closest Batouri hotel is Batouri.
25 Closest Fortuna hotel is F

203 Closest Mackenzie hotel is Mackenzie Airport (YZY).
204 Closest Ilulissat hotel is Ilulissat.
205 Closest Tual hotel is Tual.
206 Closest Bouaké hotel is Bouake.
207 Closest Shimanovsk hotel is Shimanovsk.
208 Closest Ancud hotel is Ancud.
Missing field...skipping 209 Adrar
210 Closest Portland hotel is Portland.
211 Closest Bathsheba hotel is Wilson Hill.
212 Closest ‘Āhuimanu hotel is Honolulu.
213 Closest Korem hotel is Korem.
214 Closest Cidreira hotel is CIDREIRA.
215 Closest Nandikotkūr hotel is Nandikotkur.
216 Closest Sarangani hotel is Municipality of Sarangani.
217 Closest Ikongo hotel is Fort-Carnot.
218 Closest La Ronge hotel is La Ronge.
219 Closest Alakurtti hotel is Alakurtti.
220 Closest Ballina hotel is Ballina.
221 Closest Port Hedland hotel is Port Hedland.
222 Closest Pathein hotel is Pathein.
223 Closest Sitka hotel is Sitka.
224 Closest Lafiagi hotel is Lafiagi.
225 Closest Turaif hotel is Turaif.
226 Closest Kaitangata hotel is Kaitangata.
227 Closest Kavarat

407 Closest Lompoc hotel is Lompoc.
408 Closest Tucupita hotel is Tucupita.
409 Closest Bobo-Dioulasso hotel is Bobo-Dioulasso.
410 Closest Buala hotel is Buala.
411 Closest Isangel hotel is Isangel.
412 Closest Alexandria hotel is Alexandria.
413 Closest Novodugino hotel is Novodugino.
414 Closest At-Bashi hotel is At-Bashi.
415 Closest Danjiangkou hotel is Xiangyang.
416 Closest Santiago del Estero hotel is Santiago del Estero.
417 Closest Akdağmadeni hotel is Akdağmadeni.
418 Closest Kota Bharu hotel is Kota Bharu.
419 Closest Eston hotel is Eston.
420 Closest Jalu hotel is Jalu.
421 Closest Payo hotel is Keboer.
422 Closest Sandpoint hotel is Sandpoint.
423 Closest Nizhniy Tsasuchey hotel is Nizhnii Tsasuchei.
424 Closest Egvekinot hotel is Egvekinot.
425 Closest Kamen'-na-Obi hotel is Kamen-na-Obi.
426 Closest Novy Urengoy hotel is Novy Urengoy.
427 Closest Mattru hotel is Bandajuma.
428 Closest Manokwari hotel is Manokwari.
429 Closest Qasigiannguit hotel is Qasigiannguit.
430 Cl

In [12]:
hotel_df.head(50)

,City,Lat,Lng,Max temp,Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
0,Kavieng,-2.5744,150.7967,80.69,80.69,82,100,7.56,PG,1639820794,Kavieng
1,Saint-Philippe,-21.3585,55.7679,86.11,83.97,72,16,2.01,RE,1639821072,Saint-Philippe
2,Yulara,-25.2406,130.9889,109.24,109.24,8,15,10.36,AU,1639820871,Petermann
3,Hermanus,-34.4187,19.2345,89.91,78.80,42,10,13.42,ZA,1639821073,Hermanus
4,Khatanga,71.9667,102.5000,-39.32,-39.32,100,65,3.13,RU,1639820909,Chatanga
5,Trstená,49.3610,19.6125,35.20,31.98,87,89,7.00,SK,1639821073,Trstená
6,Takaungu,-3.6837,39.8566,87.89,87.89,65,89,7.07,KE,1639821073,Kilifi
7,Mataura,-46.1927,168.8643,57.47,57.47,67,100,2.21,NZ,1639821074,Gore
8,Saint Paul Harbor,57.7900,-152.4072,30.90,30.90,76,100,5.48,US,1639820826,Kodiak
9,Newport,51.5877,-2.9984,47.55,46.36,90,61,11.92,GB,1639821054,Newport


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

# Add marker layer on top of heat map
# Display figure

In [14]:
# Create a marker_layer using the hotel_info to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)                        
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))